# LOADING DATASET

In [38]:
import pathlib
import urllib
import zstandard
import chess
import torch
import numpy as np
from torch import nn

In [39]:
def __download(url: str, name: str) -> str:
    path, _ = urllib.request.urlretrieve(url, name)
    return path


def __unpack(path: str, name: str):
    input_file = pathlib.Path(path)
    with open(input_file, 'rb') as compressed:
        decomp = zstandard.ZstdDecompressor()
        output_path = name
        with open(output_path, 'wb') as destination:
            decomp.copy_stream(compressed, destination)
            destination.close()
        compressed.close()


def __remove(path: str):
    pathlib.Path.unlink(pathlib.Path(path))

In [40]:
path = __download("https://database.lichess.org/lichess_db_puzzle.csv.zst", "lichess_db_puzzle.csv.zst")

In [41]:
__unpack(path, "lichess_db_puzzle.csv")

In [42]:
__remove("lichess_db_puzzle.csv.zst")

In [43]:
class Puzzle:
    def __init__(self, row: str):
        fields = row.split(',')
        self.fen = fields[1]
        self.moves = fields[2].split(" ")
        self.tags = fields[7].split(" ")

    def __str__(self):
        return "{fen: " + self.fen + " ,tags: [" + ", ".join(self.tags) + "],moves: [" + ",".join(self.moves) + "]}"

In [44]:
def load(k: int) -> [Puzzle]:
    f = open("lichess_db_puzzle.csv")
    f.readline()
    result = []
    for i in range(k):
        result.append(Puzzle(f.readline()))
    f.close()
    return result

In [45]:
load(10)[0].__str__()

'{fen: r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - - 0 24 ,tags: [crushing, hangingPiece, long, middlegame],moves: [f2g3,e6e7,b2b1,b3c1,b1c1,h6c1]}'

# FILTER DATASET

In [46]:
expected_tags = {
    'attraction',
    'discoveredAttack',
    'doubleCheck',
    'fork',
    'pin',
    'sacrifice',
    'skewer',
    'xRayAttack',
    'zugzwang',
    'deflection',
    'clearance'
}

In [47]:
expected_tags_list = list(expected_tags)

In [48]:
def filter_data(data: [Puzzle]) -> [Puzzle]:
    return list(filter(lambda p: len(set(p.tags) & expected_tags) == 1, data))

In [49]:
len(filter_data(load(100)))

37

# CONVERSION TO TENSOR

In [50]:
def bitboard_to_tensor(bitboard: int) -> torch.Tensor:
    li = [1 if digit == '1' else 0 for digit in bin(bitboard)[2:]]
    li = [0 for _ in range(64 - len(li))] + li
    return torch.tensor(li).reshape((8, 8))

In [51]:
def fen_to_tensors_list(fen: str) -> [torch.Tensor]:
    board = chess.Board(fen)
    return [
        bitboard_to_tensor(board.occupied_co[chess.WHITE]),
        bitboard_to_tensor(board.occupied_co[chess.BLACK]),
        bitboard_to_tensor(board.pawns),
        bitboard_to_tensor(board.kings),
        bitboard_to_tensor(board.queens),
        bitboard_to_tensor(board.knights),
        bitboard_to_tensor(board.bishops),
        bitboard_to_tensor(board.rooks)
    ]

In [52]:
fen_to_tensors_list(load(1)[0].fen)

[tensor([[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 1, 0, 1, 0],
         [1, 1, 0, 0, 0, 1, 0, 1],
         [1, 0, 0, 0, 0, 0, 0, 0]]),
 tensor([[1, 0, 0, 0, 0, 0, 0, 1],
         [1, 0, 0, 1, 0, 0, 1, 1],
         [0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]]),
 tensor([[0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 1, 1],
         [0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [1, 1, 0, 0, 0, 1, 0, 1],
         [0, 0, 0, 0, 0, 0, 0, 0]]),
 tensor([[1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0

In [53]:
def move_to_tensor(move: str) -> torch.Tensor:
    x1 = 7 - ord(move[0]) + ord('a')
    y1 = 8 - int(move[1])
    x2 = 7 - ord(move[2]) + ord('a')
    y2 = 8 - int(move[3])
    tensor = torch.zeros(8, 8)
    tensor[y1][x1] = 1
    tensor[y2][x2] = 1
    return tensor

In [54]:
print(move_to_tensor('e2e4'))

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


In [55]:
def puzzle_to_tensor(puzzle: Puzzle) -> torch.Tensor:
    fen_tensors = fen_to_tensors_list(puzzle.fen)
    move_tensors = [move_to_tensor(puzzle.moves[0]), move_to_tensor(puzzle.moves[1])]  # FIRST TWO MOVES
    return torch.stack(fen_tensors + move_tensors)

In [56]:
puzzle_to_tensor(load(1)[0])

tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 1., 0., 1., 0.],
         [1., 1., 0., 0., 0., 1., 0., 1.],
         [1., 0., 0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 1.],
         [1., 0., 0., 1., 0., 0., 1., 1.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 1., 1.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0., 1., 0., 1.],
       

# CONVERT AND BATCH DATASET

In [57]:
def puzzle_to_truth(puzzle: Puzzle) -> torch.Tensor:
    tensor = torch.zeros(len(expected_tags_list))
    [tag] = set(puzzle.tags) & expected_tags
    index = expected_tags_list.index(tag)
    tensor[index] = 1
    return tensor

In [58]:
puzzle_to_truth(filter_data(load(100))[0])

tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [59]:
BATCH_SIZE = 64

In [60]:
def convert_dataset(puzzles: [Puzzle]) -> list[tuple[torch.Tensor, torch.Tensor]]:
    return [(puzzle_to_tensor(puzzle), puzzle_to_truth(puzzle)) for puzzle in puzzles]

In [85]:
def dataset_to_batches(dataset: list[tuple[torch.Tensor, torch.Tensor]]) -> list[tuple[torch.Tensor, torch.Tensor]]:
    batches = []
    index = 0
    while index + BATCH_SIZE <= len(dataset):
        batch = []
        truth = []
        max_index = index + BATCH_SIZE
        while index < max_index:
            batch.append(dataset[index][0])
            truth.append(dataset[index][1])
            index += 1
        batches.append((torch.stack(batch).cuda(), torch.stack(truth).cuda()))

    return batches

In [86]:
batched_dataset=dataset_to_batches(convert_dataset(filter_data(load(10000))))
print(len(batched_dataset))
print(batched_dataset[0][0].shape,batched_dataset[0][1].shape)

47
torch.Size([64, 10, 8, 8]) torch.Size([64, 11])


# TRAIN

In [157]:
class Model(nn.Module):
    def __init__(self, *args, **kwargs):
        super(Model, self).__init__()
        self.classifier = nn.Sequential(*args, **kwargs)

    def forward(self, X):
        return self.classifier.forward(X)


model = Model(nn.Conv2d(10, 8 * 8, kernel_size=4, padding=2),
              nn.ReLU(),
              nn.Conv2d(8 * 8, 4 * 4, kernel_size=2, padding=2),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=4, stride=1),
              nn.Conv2d(4*4, 8 * 8, kernel_size=2, padding=2),
              nn.ReLU(),
              nn.Conv2d(8 * 8, 1, kernel_size=4, padding=2),
              nn.ReLU(),
              nn.Flatten(),
              nn.Linear(169, 256),
              nn.ReLU(),
              nn.Linear(256, 64),
              nn.ReLU(),
              nn.Linear(64, 11),
              nn.Softmax(),
              )
criterion = (
    nn.CrossEntropyLoss()
)


In [158]:
size_to_load=1000000
test_batches_count=200

In [159]:
all_batches=dataset_to_batches(convert_dataset(filter_data(load(size_to_load))))
train_batches=all_batches[test_batches_count:]
test_batches=all_batches[:test_batches_count]
print(len(all_batches),len(train_batches),len(test_batches))

4770 4570 200


In [160]:
def train(model, criterion, optimizer, epoch):
    model.cuda()
    criterion.cuda()
    batches = train_batches
    print("Dataset size:", len(batches))
    batch_index = 0
    for i in range(epoch):

        batch = batches[batch_index][0]
        truth = batches[batch_index][1]

        if batch_index == len(batches):
            batch_index = 0

        optimizer.zero_grad()
        out = model.forward(batch)
        loss = criterion(out, truth)
        loss.backward()
        optimizer.step()

        if i % 1000 == 0:
            print(loss.item())

In [166]:
train(model,
      criterion,
      torch.optim.SGD(model.classifier.parameters(), lr=0.001),
      100000)

Dataset size: 4570
2.345280408859253
2.3385794162750244
2.3307228088378906
2.3214187622070312
2.3103208541870117
2.297081708908081
2.2814924716949463
2.2637104988098145
2.2446703910827637
2.225867748260498
2.2086737155914307
2.1945362091064453
2.1819605827331543
2.1690762042999268
2.1599607467651367
2.156102418899536
2.1545300483703613
2.153789520263672
2.1533894538879395
2.153149366378784
2.1529932022094727
2.152885913848877
2.15280818939209
2.152750015258789
2.152705192565918
2.152669668197632
2.1526412963867188
2.1526176929473877
2.1525983810424805
2.1525816917419434
2.1525676250457764
2.1525557041168213
2.152545213699341
2.152535915374756
2.1525278091430664
2.1525206565856934
2.1525139808654785
2.152508497238159
2.152503252029419
2.152498483657837
2.152494192123413
2.1524903774261475
2.152486801147461
2.1524834632873535
2.1524806022644043
2.152477741241455
2.152475118637085
2.152472734451294
2.152470588684082
2.15246844291687
2.1524665355682373
2.1524646282196045
2.152462959289551


In [167]:
def test(model, criterion):
    model.cuda()
    criterion.cuda()
    batches = test_batches
    print("Dataset size:", len(batches))
    batch_index = 0
    
    total_loss = 0
    for i in range(len(batches)):

        batch = batches[batch_index][0]
        truth = batches[batch_index][1]

        if batch_index == len(batches):
            batch_index = 0

        out = model.forward(batch)
        loss = criterion(out, truth)
        total_loss += loss.item()

    return total_loss / len(batches)

In [168]:
test(model, criterion)

Dataset size: 200


2.1680612564086914

In [182]:
def eye_test(model,puzzle):
    tensor=puzzle_to_tensor(puzzle).cuda()
    out=model.forward(tensor)
    return sorted(zip(expected_tags_list,out.squeeze().tolist()),key=lambda x:-x[1]),puzzle.tags

eye_test(model,filter_data(load(100))[0])

([('fork', 0.9998440742492676),
  ('pin', 5.93155637034215e-05),
  ('sacrifice', 3.174515222781338e-05),
  ('discoveredAttack', 1.6470301488880068e-05),
  ('deflection', 1.1752566024370026e-05),
  ('clearance', 7.617819846927887e-06),
  ('zugzwang', 7.274342351593077e-06),
  ('attraction', 6.431613201129949e-06),
  ('skewer', 6.2638996496389154e-06),
  ('doubleCheck', 5.9139306358702015e-06),
  ('xRayAttack', 3.1618544653611025e-06)],
 ['crushing', 'endgame', 'exposedKing', 'long', 'skewer'])